In [4]:
#pip install opencv-python

In [5]:
import os
from pathlib import Path
import cv2 

In [ ]:
ROOT = Path("/mnt/d/TFM/German dataset/FullIJCNN2013") 
GT_FILE = ROOT / "gt.txt"

# Guardadas temporalmente dentro de la carpeta del dataset alemán hasta que las traslademos a su carpeta
LABELS_DIR = ROOT / "labels_yolo"
LABELS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Leer gt.txt

# El archivo gt.txt tiene una estructura un poco diferente a la que necesitamos para YOLO.
# En gt.txt tenemos una lista de todas las señales existentes dentro de las imágenes del conjunto de datos
annotations = {}  # { "00000.ppm": [ (left,top,right,bottom,class_id), ... ] }

with GT_FILE.open("r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        img_name, left, top, right, bottom, class_id = line.split(";")
        left, top, right, bottom, class_id = map(int, [left, top, right, bottom, class_id])
        annotations.setdefault(img_name, []).append((left, top, right, bottom, class_id))

print(f"Imágenes con anotaciones: {len(annotations)}")

Imágenes con anotaciones: 741


In [ ]:
# Conversión a YOLO

# Para YOLO lo que necesitamos es un archivo txt por imagen con las señales que se pueden reconocer en la misma
# Por lo que tenemos que reestructurar toda esta información y crear un archivo por imagen
#   donde aparecerán tantas líneas como señales de tráfico haya en la fotografía
# También algo a tener en cuenta es que YOLO también necesita que las coordenadas de las cajas estén normalizadas
for img_name, ann_list in annotations.items():
    img_path = ROOT / img_name
    if not img_path.exists():
        print(f"[AVISO] No se encuentra la imagen: {img_path}")
        continue

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[AVISO] OpenCV no puede leer: {img_path}")
        continue

    h, w = img.shape[:2]

    yolo_lines = []
    for (left, top, right, bottom, class_id) in ann_list:
        # Asegurar que las coordenadas están dentro de la imagen
        left = max(0, min(left, w - 1))
        right = max(0, min(right, w - 1))
        top = max(0, min(top, h - 1))
        bottom = max(0, min(bottom, h - 1))

        # Centro y tamaño
        cx = (left + right) / 2.0
        cy = (top + bottom) / 2.0
        bw = (right - left)
        bh = (bottom - top)

        # Normalizar
        cx /= w
        cy /= h
        bw /= w
        bh /= h

        yolo_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    # Guardar fichero de etiquetas
    label_path = LABELS_DIR / (Path(img_name).stem + ".txt")
    with label_path.open("w") as lf:
        lf.write("\n".join(yolo_lines))

print("Conversión a formato YOLO terminada.")

Conversión a formato YOLO terminada.
